In [49]:
# 라이브러리 불러오기
from selenium import webdriver
from bs4 import BeautifulSoup
import time

from tqdm.notebook import tqdm # 상태진행바
import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import json

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import re
from datetime import datetime

#excel control library, ! pip install openpyxl
from openpyxl import Workbook

In [50]:
# 날짜 데이터 parsing
def get_date(date):
    date = date.text if date else ''
    # 정규 표현식 패턴 정의
    pattern = r'(\d{4})년 (\d+)월 (\d+)일 \S+'

    # 검색 실행
    match = re.search(pattern, date)

    # 결과 처리 및 형식 변경
    if match:
        year, month, day = match.groups()  # 연, 월, 일을 추출합니다.

        # 날짜 객체 생성
        date_obj = datetime(int(year), int(month), int(day))

        # 지정된 형식으로 날짜 문자열 포맷
        formatted_date = date_obj.strftime('%Y-%m-%d')

        return(formatted_date)
    else:
        return("None")

In [51]:
def get_review(url):
    
    #excel파일 생성
    xlsx = Workbook()
    del xlsx['Sheet']
    list_sheet = xlsx.create_sheet('output')
    list_sheet.append(['nickname', 'content', 'date', 'revisit'])
    #url 접속
    service = Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(service=service)
    browser.get(url)
    
    try:
        #리뷰페이지 접속
        menu_list = WebDriverWait(browser, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.tpj9w._tab-menu')))
        if len(menu_list) > 2:
            menu_list[2].click()  # Click on the review tab
        else:
            print("Review tab not found or fewer tabs than expected.")
            return
        #가게이름 추출
        store_name = browser.find_element(By.CLASS_NAME, "Fc1rA")
        store_name = store_name.text
        print(store_name)
        #더보기 끝까지 클릭
        try:
            while True:
                more_button = WebDriverWait(browser, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.fvwqf')))
                more_button.click()
                time.sleep(0.4)  # Moderate delay to simulate user behavior
        except Exception as e:
            print('Click 더보기 finish')
    
        # 닉네임, 내용, 날짜, 재방분 크롤링
        html = browser.page_source
        bs = BeautifulSoup(html, 'lxml')
        reviews = bs.select('li.owAeM')

        for r in reviews:
            nickname = r.select_one('div.qgLL3')
            content = r.select_one('span.zPfVt')
            date = r.select('span.CKUdu')[0]
            revisit = r.select('span.CKUdu')[1]

            # exception handling, date는 get_date()에서 에러처리와 파싱 같이함.
            nickname = nickname.text if nickname else ''
            content = content.text if content else ''
            date = get_date(date)
            revisit = revisit.text if revisit else ''
            time.sleep(0.06)

            print(nickname, '/', content, '/', date, '/', revisit)
            list_sheet.append([nickname, content, date, revisit])
            time.sleep(0.06)
            
        file_name = 'naver_review_' + store_name + '.xlsx'
        xlsx.save(file_name)
        print("hello")

    except Exception as e:
        print("error !!", str(e))

    finally:
        browser.quit()

In [52]:
url = "https://m.place.naver.com/restaurant/1083519696/home?entry=pll"

In [53]:
get_review(url)

하이레
Click 더보기 finish
kan**** / 일요일 게릴라 오픈때 방문했는데 맛있었어요 구성과 곁들임이 다양해서 여러 시도를 할 수 있고, 친절하고 섬세하신 사장님덕분에 편안하고 배부르게 기분좋은 식사 했습니다. 다음엔 백석으로! ◡̈  / 2024-03-03 / 1번째 방문
이동네먹짱 / 먹어본 돈가스 중에 최고예요. 광운대 앞 한줄기 빛..✨ 부드럽고, 촉촉하고, 잡내 전혀 없어서 진짜 게 눈 감추듯 순삭 했어요. 고기에 찍어먹을 소스가 다양해서 하나씩 찍어먹어보는 재미도 있어요. 고
기가 너무너무 맛있어서 그냥 말돈 소금 살짝 찍어서 고기 맛 그대로 느끼며 먹는 게 제일 맛있더라고요. 식당 내부는 일본에 온 것처럼 일본 상품이 진열되어 있고, 집기류가 아기자기해요. 히레가스, 특
등심 다 맛있었어요. 맛있다는 일식돈가스집 많이 가봤는데 여기만큼 촉촉하고 맛있는 곳은 처음이에요. 자주자주 가고 싶어요. 단골 예약입니다. 진짜 강추해요!! / 2024-01-22 / 1번째 방문
붉은지렁이70 / 꼭 카레 추가해서 드세요!!
정말 맛있어요!! / 2024-04-15 / 3번째 방문
ksr**** / 근처 식물샵 놀러온 겸 들리게되었습니다 ㅎㅎ 근래 먹어본 돈까스 중 제일 맛있었습니다. 매장 곳곳 식물과 오브젝트 등 사장님께서 신경을 많이 쓰신 게 느껴졌고 고기 질을 비롯하여 튀김의 두께, 스끼다시 등 조화가 푸짐하고 맛있었습니다 다음 번에도 또 방문하겠습니다. 번창하세요 ㅎㅎ / 2024-02-13 / 1번째 방문
붉은지렁이70 / 노원구 돈카츠 1티어는 여기가 맞아요!! / 2024-03-25 / 2번째 방문
오늘은 돈카츠 / 카테고리를 5개 밖에 고르지 못하는 게 아쉽습니다…🥹 광운대학교 학생들이 너무 부러운 맛이구요… 크리스마스 최고의 선물임이 틀림 없습니다!!🫶 반찬 구성은 밥 부터 디저트까지 말할 것도 없고 
사장님의 따뜻하고 친절하고 다정한 응대가 맛을 더 극상까지 끌어올리는 맛이라고 생각합니다🤦겨자, 와사비, 소금, 소스 등 다양한 선택